In [1]:
# Importing the libraries.
import time
import psutil

from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.embeddings import OllamaEmbeddings

from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [2]:
# Checker function if ollama is running fine or not.
def check_ollama(process_name):
    running = False
    for proc in psutil.process_iter(["name"]):
        if process_name in proc.info["name"]:
            running = True
            break
    return running

ollama_running = check_ollama("ollama")

if not ollama_running:
    raise RuntimeError("Ollama not running.")

print(f"Ollama is running fine: {ollama_running}")

Ollama is running fine: True


In [3]:
# Defining the LLMs which will be used for our use-case.
def create_llm():
  tool_llm = ChatOllama(
          model="llama3-groq-tool-use",
          temperature=0,
  )

  response_llm = ChatOllama(
          model="llama3.1:8b",
          temperature=0.5,
  )
  return tool_llm, response_llm

In [4]:
# Dummy function to check if payment was successful
def payment_done():
    return True
# Function which will be called if tool calling happens.
@tool
def book_room(name, contact_number, check_in_date, check_out_date, room_type)-> str:
    """Book a hotel's room on behalf of the user."""
    if(payment_done()):
        # update the DB that a room is booked.
        return f"Thanks {name}. Your {room_type} room has been booked and details will be shared to your contact number: {contact_number}."
    return "Payment was not successful. Room wasn't booked."

In [5]:
# Setting up the system prompt and basic flow of how query will be taken.
def chat(query, tool_llm, response_llm):
    messages = [HumanMessage(query)]
    ai_msg = tool_llm.invoke([SystemMessage(
                               content="""You are an AI assistant for The Lalit Hotel. 
                                            There are five types of rooms available there:
                                            - Deluxe Rooms
                                            - Premier Rooms
                                            - Executive Club Rooms
                                            - Executive Suites
                                            - Luxury Suites
                                            - Presidential Suite
                                            - The Lalit Legacy Suite
                                        Your primary functions are:
                                        Book rooms using the available booking tool
                                        Answer general questions about the hotel
                                        When booking rooms:
                                            Use the "book_room" function to check availability and make reservations
                                            Confirm details like dates, room type, and number of guests
                                            Provide booking confirmation numbers
                                        For general inquiries (You are also provided with all the information related to hotel before check that also):
                                            Answer questions about amenities, services, policies, and local attractions
                                            If you don't have specific information, politely say so and offer to connect the guest with hotel staff
                                            Always be polite, professional, and helpful. If you need more information to assist a guest, ask clarifying questions.
                                        """
                            )]
                          + messages)
    
    messages.append(ai_msg)
    
    for tool_call in ai_msg.tool_calls:
        selected_tool = {"book_room": book_room}[tool_call["name"].lower()]
        tool_msg = selected_tool.invoke(tool_call)
        messages.append(tool_msg)
        
    system_message = f"""You are a secondary language model to help the guests of THE LALIT HOTEL and responsible for reviewing the response 
    generated by the primary model and finalizing the output based on the query and the tools invoked.
    There are five types of rooms available there:
                                            - Deluxe Rooms
                                            - Premier Rooms
                                            - Executive Club Rooms
                                            - Executive Suites
                                            - Luxury Suites
                                            - Presidential Suite
                                            - The Lalit Legacy Suite
    Suggest these rooms to the user.
    Instructions:

    You will receive a query and a response generated by the first LLM.
    Your task is to review the response in the context of the original query and the actions 
    performed and provide a final, coherent response.
    If the first LLM invoked tools related to booking of rooms, ensure that the response accurately reflects 
    the results of those tool invocations.
    
    If the user's query is a generic question or greeting (e.g., "Hello", "How are you?", "Whats up?"), respond in a polite, 
    friendly, and conversational manner without using any tools.
    Ensure that the final response is clear, polite, and aligned with the user's expectations.
    Avoid stating that tools are inaccessible or unavailable unless explicitly required by the query.
    Ensure that you return a response that is understandable and appropriate for the user's request.
    Your response will be shown to the user, so DON'T PROVIDE GIBBERISH IN THE RESULT.
    """
    
    chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    MessagesPlaceholder(variable_name="conversation")
    ])

    formatted_messages = chat_prompt.format_messages(
    conversation=messages  
    )

    response = response_llm.invoke(formatted_messages)

    if ai_msg.tool_calls!=[]:
        print("------------------------------------------------------------------")
        print("Tool was called with these arguments:")
        print(ai_msg.tool_calls)
        print("------------------------------------------------------------------")
    
    if response.content=="":
      print("Final response:", ai_msg.content)
    
    else:
      print("Final response:", response.content)

In [6]:
# Start the conversation with assistant.
def start_llm(tool_llm, response_llm):
    query = str(input("Query: "))
    if(query=="bye"):
        return "Alright, bye."
    start = time.time()
    chat(query = query, tool_llm = tool_llm, response_llm = response_llm)
    end = time.time()
    print(f"Time taken to respond to this query: {end-start}.")
    start_llm(tool_llm = tool_llm, response_llm = response_llm)

In [7]:
tools = [book_room]
tool_llm, response_llm = create_llm()
tool_llm = tool_llm.bind_tools(tools)

In [ ]:
start_llm(tool_llm, response_llm)

Query:  book a deluxe suite, my name is sarthak and my contact number is 432513, i'll stay from 7th march to 10th march'25.


------------------------------------------------------------------
Tool was called with these arguments:
[{'name': 'book_room', 'args': {'check_in_date': '2025-03-07', 'check_out_date': '2025-03-10', 'contact_number': 432513, 'name': 'Sarthak', 'room_type': 'Deluxe Suite'}, 'id': '91739030-7f7e-4b52-8068-a5cd4c541aa2', 'type': 'tool_call'}]
------------------------------------------------------------------
Final response: Based on the query, I have invoked the "book_room" tool with the required parameters - check-in date (7th March '25), check-out date (10th March '25), contact number (432513), name (Sarthak) and room type (Deluxe Suite). The response is clear and concise, providing confirmation of the booking.
Time taken to respond to this query: 53.145819902420044.


Query:  hello, how are you?


Final response:  We have a lovely hotel here at The Lalit, with various rooms and suites to suit your needs. Are you looking for some recommendations or information about our accommodations?
Time taken to respond to this query: 1.3176093101501465.


Query:  how many types of rooms are there in your hotel?


In [ ]:
# book a deluxe suite, my name is sarthak and my contact number is 432513, i'll stay from 7th march to 10th march'25.